In [4]:
import torch
test = torch.load('./data/models/decoder-0.pkl', map_location='cpu')

In [6]:
type(test)

collections.OrderedDict

In [1]:
from data_loader import get_loader
from build_vocab import Vocabulary
from torchvision import transforms
import pickle

train_annote_path = './data/annotations/train/captions.json'
train_image_path = './data/images/train'
batch_size = 5
with open('./data/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
transform = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

data_loader = get_loader(train_annote_path, train_image_path, vocab, transform,
                        batch_size,shuffle=True, num_workers=2)

In [2]:
for (images, captions, lengths) in data_loader:
    break

print(captions[0, 1:])
captions[0]

tensor([  4, 152,  62, 163,   4, 285, 232,   8,   4,  42,  17,   4, 104,  12,
          2])


tensor([  1,   4, 152,  62, 163,   4, 285, 232,   8,   4,  42,  17,   4, 104,
         12,   2])

In [3]:
from model import DecoderLSTM

vocab_size = len(vocab)
embed_size=512
feature_num=512
hidden_size=512
attention_dim=196
num_layers=2

decoder = DecoderLSTM(vocab_size, embed_size, feature_num, hidden_size, 
                      attention_dim, num_layers)

In [4]:
from model import EncoderCNN
encoder = EncoderCNN()
features = encoder(images)
print(features.shape)

torch.Size([5, 512, 196])


In [5]:
output, alphas = decoder(features, captions, lengths)
print(output.shape, alphas.shape)

RuntimeError: input must have 3 dimensions, got 2

In [11]:
import torch.nn as nn
from model import Attention

embed = nn.Embedding(vocab_size, embed_size)
attention = Attention(feature_num, hidden_size, attention_dim, num_layers)
f_beta = nn.Linear(hidden_size*num_layers, feature_num)
sigmoid = nn.Sigmoid()
lstm = nn.LSTM(embed_size + feature_num, hidden_size, num_layers, batch_first=True)
# input/output: (batch, seq, feature)
dropout = nn.Dropout(p=0.5)
linear = nn.Linear(hidden_size, vocab_size)

In [6]:
def init_hidden(batch_size=5):
    h0 = torch.zeros(num_layers, batch_size, hidden_size)
    c0 = torch.zeros(num_layers, batch_size, hidden_size)

    if torch.cuda.is_available():
        h0 = h0.cuda()
        c0 = c0.cuda()

    return h0, c0

In [8]:
import torch
batch_size = features.shape[0]
h, c = init_hidden(batch_size)

embedding = embed(captions) # size: caption_len*embed_size
decode_lengths = [len - 1 for len in lengths]

predictions = torch.zeros(batch_size, max(decode_lengths), vocab_size)
alphas = torch.zeros(batch_size, max(decode_lengths), attention_dim)

print(predictions.shape, alphas.shape)

torch.Size([5, 15, 2699]) torch.Size([5, 15, 196])


In [12]:
from model import Attention
import torch
attention = Attention(feature_num, hidden_size, attention_dim, num_layers)
h, c = decoder.init_hidden(batch_size=5)

i = 0
batch_idx_to_decode = sum([l > i for l in lengths])
z, alpha = attention(features[:batch_idx_to_decode], h[:batch_idx_to_decode])
h_gate = h[:batch_idx_to_decode, :, :]
h_gate = torch.flatten(h_gate.view(h_gate.shape[1], h_gate.shape[0], -1), start_dim=1, end_dim=2)
print(z.shape, h_gate.shape)
z = z * sigmoid(f_beta(h_gate))

torch.Size([5, 512]) torch.Size([5, 1024])


In [18]:
embedding[:batch_idx_to_decode, i, :].shape

torch.Size([5, 512])

In [14]:
lstm_input = torch.cat([embedding[:batch_idx_to_decode, i, :], z[:batch_idx_to_decode, :]], dim=1)

In [17]:
print(h.shape)
lstm_input.shape

torch.Size([2, 5, 512])


torch.Size([5, 1024])